$S(x,t)$ is sampled at $M$ positions along the conveyor belt periodically for $N$ time intervals $\Delta t$   $S^n_m=S(x_m,t_n)$ 

The height of the sandpile at the beginning of the conveyor belt is $H_0(t)=0$  <em>(Dirichlet)</em>

The rate at which the sand falls of the end of the conveyor belt is proportional to the height of the sand pile? $\partial_x h(L,t)=\alpha h(L,t)$ <em>(Robin)</em>

$$
\begin{align}
AH^{n+1} &= b\quad \text{where}\quad b=H^n + S^n\Delta t \tag{1}\\
H_M^{n+1} &= (2\alpha \Delta x) H_{M-1}^{n+1} + H_{M-2}^{n+1}\\
H_0 &= 0 
\end{align}
$$

Assume that sand is dispensed at the beginning of the conveyor belt. 
Sand is poured into a funnel from another process from above, and falls through the funnel onto the beginning of the conveyor belt to be transported and subsequently processed.
Assume that the lower end of the funnel measures 15cm in diamter. 
$$
S(x,t) = 
\begin{cases}
0.02 & \text{if } 0<x\leq 0.15 \\
0  & \text{otherwise}
\end{cases}
$$


In [29]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse        import diags_array
from scipy.sparse.linalg import spsolve

In [30]:
L= 10.0 # 10m convyer belt, longest possible
v= 0.375 # advection velocity, mid point of suggested speeds
D= 0.02 # diffusion coefficient, small spreading of sand

s = 0.02 # source term, cont. rate of sand being added
s_lim = 0.15 

Nx = 200; Nt = 200
T = 40 # simulation length in seconds

x = np.linspace(0, L, Nx+1)
dx = x[1] - x[0] # grid spacing for finite differences

t = np.linspace(0, T, Nt+1)
dt = t[1] - t[0]

We solve the tridiagonal system in equation (1)

In [31]:
C = (D*dt)/(dx**2)
U = (v*dt)/dx

In [32]:
# Store data in sparse format to prevent the memory allocation of M² - M + 2 zeros
Nx=10

# Values on the three non-zero diagonals
diags = np.ones((3,Nx+1))

# Add 2C to main diagonal (first index = 0 represents the main diagonal here)
diags[0] += 2 * C + U          

# The upper and lower diagonals are -C except in the first/last row, which we will deal with next
diags[1] = - 1 - C - U
diags[2] = - 1 - C

# Fix the upper diagonal in the first row (the first entry should be -2C)
diags[1,0] = 0

# Fix the lower diagonal in the last row (the penultimate entry should be -2C - the last is ignored when making A)
diags[2,-2] = 0

# Create the matrix, taking care of the order in which we defined the diagonals
# format = 'csr' defines a sparse matrix
A = diags_array(diags, offsets = (0,1,-1), shape = (Nx+1, Nx+1), format = 'csr')

In [ ]:
# ** Initialize output ** #

# Preallocate the numerical solution in a matrix: 
# the first dimension is time and the second is space; i.e., each row corresponds to a timestep
H = np.zeros((N+1,M+1))
S = np.zeros((N+1,M+1))

H[0] = 0
# Enforce boundary conditions for all space points
H[:,0] = 0
H[:,]

for i in range(Nx+1):
    if(i*dx < s_lim) S[:,i] = s

# Also store exact solution
#U_ex = np.zeros((N+1,M+1))

# ** Iterate ** #
for n in range(0, N):
    # Solve linear system instead of computing the inverse: 
    # computing the inverse is expensive and numerically instable 
    # SciPy has a subroutine for sparse matrices, but if you want to stick to NumPy, 
    # you can instead use: np.linalg.solve(A.toarray(),U[n])
    U[n+1] = spsolve(A, U[n] + S[n])
    #U_ex[n] = U_exact_1(x,t[n])

# Feel free to compare the solution as before:
#plotting(U_ex,x,t, U_exact_1)